In [2]:
import pandas as pd

result = pd.read_csv("result.csv")
del result['Unnamed: 0']
result.head()

,tx_date,tx_count,Tx_volume_Ether,Tx_volume($),tx_count_ERC20,Tx_volume_ERC20,Tx_volume_ERC20($),Volume($),Market Cap($),DAU,CUM
0,2016-01-01,8233,3.445478e+05,326639.542431,0.0,0.0,0.0,206062,71980386,8006,8006
1,2016-01-02,9164,2.592400e+05,242940.035122,0.0,0.0,0.0,255504,71176658,8814,16820
2,2016-01-03,9258,3.708200e+05,360401.805781,0.0,0.0,0.0,407632,73843292,9019,25839
3,2016-01-04,9474,1.541255e+05,147109.680450,0.0,0.0,0.0,346245,72543707,9059,34898
4,2016-01-05,16430,1.036291e+06,984659.012911,0.0,0.0,0.0,219833,72240974,11164,46062


In [3]:
final = pd.read_csv("final.csv")
del final['Unnamed: 0']
final.set_index("x_lst")

,Coef,Score
x_lst,,
DAU,1.327946e+05,0.848060
tx_count,7.853991e+04,0.815805
Volume($),1.784916e+01,0.700871
Tx_volume($),5.828936e+00,0.433478
tx_count_ERC20,8.718415e+04,0.361827
CUM,1.976944e+02,0.271924
Tx_volume_Ether,7.874096e+02,0.033819
Tx_volume_ERC20,-3.055525e-53,0.001385
Tx_volume_ERC20($),-7.225363e-56,0.000188


> 위 표를 통해 사실상 'DAU, tx_count, Volume($)'를 제외하고 나머지 값들은 거의 의미 없다고 볼 수 있다

In [4]:
import plotly.plotly as py
import plotly.graph_objs as go

def data_to_plotly(x):
    k = []
    
    for i in range(0, len(x)):
        k.append(x[i][0])
        
    return k

from sklearn import linear_model

X = result['Tx_volume($)'].values
y = result['Market Cap($)'].values
X = X.reshape(len(X), 1)
y = y.reshape(len(y), 1)
regr = linear_model.LinearRegression()
regr.fit(X, y)

p1 = go.Scatter(x=data_to_plotly(X), 
                y=y,
                mode='markers',
                marker=dict(color='blue')
               )

layout = go.Layout(xaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   yaxis=dict(ticks='', showticklabels=False,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

fig = go.Figure(data=[p1], layout=layout)

py.iplot(fig)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



> 실제로 R2값이 0.5밑인 다른 feature들은 선형이라고 보기 어렵다 (위 그래프는 top3변수 다음으로 높았던 Tx_Volume($)와 Market Cap간의 관계)

> 각 선형회귀 분석 결과 가장 관계가 높았던 Top3인 세 변수를 x로, y를 market cap으로 다중선형회귀분석을 해본결과,

In [5]:
import statsmodels.api as sm

X = sm.add_constant(X)
model = sm.OLS(result['Market Cap($)'], result[['DAU', 'tx_count', 'Volume($)']]).fit()

predictions = model.predict()
print_model = model.summary()
print_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Market Cap($)   R-squared:                  -98618.601
Model:                            OLS   Adj. R-squared:             -98889.286
Method:                 Least Squares   F-statistic:                    -364.3
Date:                Thu, 07 Feb 2019   Prob (F-statistic):               1.00
Time:                        10:10:11   Log-Likelihood:                -26885.
No. Observations:                1096   AIC:                         5.378e+04
Df Residuals:                    1093   BIC:                         5.379e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
DAU          9.58e+04   7453.474     12.854      0.000    8.12e+04     1.1e+05
tx_count    1.586e+04   3870.728      4.097      0.000    8262.901    2.35e+04
Volume($)      0.4014      0.615      0.653      0.514      -0.805       1.608
==============================================================================
Omnibus:                      206.302   Durbin-Watson:                   0.183
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              704.581
Skew:                           0.895   Prob(JB):                    1.01e-153
Kurtosis:                       6.496   Cond. No.                     4.09e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.09e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

> R2값의 결과가 형편없음을 알 수 있다. 그렇다면 위 3 변수와 Market Cap간의 관계를 OLS로 추정하는 것은 옳지 않은 것을 알 수 있다.

> Statsmodels가 아니라 Sklearn의 linear regression을 사용하는 경우 더 R2값이 높지만, 아래에 제시되어있는 이유로 신뢰성이 높은 지표가 아니다

> PCA와 OLS의 차이 : OLS는 최소자승법으로 직선과 데이터의 거리를 최소화하는 반면, PCA는 데이터의 분산이 가장 큰 방향을 구한다
>
> 회귀분석 모형 개발 시 입력변수들간의 상관관계가 높은 다중공선성(multicollinearity)이 존재할 경우 모형이 잘못만들어지고 따라서 해석에도 문제가 생길 수 있어 이 경우 회귀분석보다 주성분분석을 통해 상관도가 높은 변수들을 하나의 주성분으로 축소하여 활용하는 것이 좋다

> 각 변수간 상관관계 분석 결과,

In [6]:
re = result[['DAU', 'tx_count', 'Volume($)']].corr(method='pearson')
print(re)

                DAU  tx_count  Volume($)
DAU        1.000000  0.963617   0.918099
tx_count   0.963617  1.000000   0.883152
Volume($)  0.918099  0.883152   1.000000


> DAU와 tx_count간의 상관관계가 1에 가깝고, tx_count와 Volume사이, DAU와 volume사이의 상관관계도 만만치 않게 높음을 알 수 있다

# 주성분분석 선형회귀

In [7]:
top3 = result[['tx_date','DAU', 'tx_count', 'Volume($)']]
top3 = top3.set_index('tx_date')
top3.head()

,DAU,tx_count,Volume($)
tx_date,,,
2016-01-01,8006,8233,206062
2016-01-02,8814,9164,255504
2016-01-03,9019,9258,407632
2016-01-04,9059,9474,346245
2016-01-05,11164,16430,219833


In [8]:
from sklearn.decomposition import PCA

In [9]:
pca = PCA(n_components=1)

In [10]:
model_pca = pca.fit(top3.values)

In [11]:
model_pca.transform(top3.values)

array([[-1.01159378e+09],
       [-1.01154434e+09],
       [-1.01139221e+09],
       ...,
       [ 2.15723018e+09],
       [ 1.64828703e+09],
       [ 1.34656045e+09]])

In [12]:
model_pca.explained_variance_ratio_

array([0.99999998])

In [13]:
X = top3
y = result['Market Cap($)']

In [14]:
from sklearn.cross_decomposition import PLSRegression

In [15]:
pls = PLSRegression(n_components=1)
pls.fit(X, y)

PLSRegression(copy=True, max_iter=500, n_components=1, scale=True, tol=1e-06)

In [16]:
pls.score(X, y)

0.832262975413609